In [3]:
import concurrent.futures

from factscore.atomic_facts import AtomicFactGenerator

generator = AtomicFactGenerator(
    "api.key", "demos", gpt3_cache_file="gpt_cache.pkl"
)

def process_data_single(dp):
    if not {"topic": dp["topic"], "num_response": dp["num_response"]} in annotated_bio:
        atomic_facts, _ = generator.run(dp["output"])
        annotations = []
        for f in atomic_facts:
            annotations.append({
                "text": f[0],
                "model-atomic-facts": [{"text": t} for t in f[1]]
            })
        dp["annotations"] = annotations

        return dp
    else:
        return None


def process_data_multi(data):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_data_single, d): d for d in data}

        concurrent.futures.wait(futures)

        for future in futures:
            annotated_dp = future.result()
            if annotated_dp is not None:
                append_dict_to_jsonl("Llama-1-7B-facts.jsonl", annotated_dp)

[nltk_data] Downloading package punkt to /home/haznitrama/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from tqdm import tqdm

annotated_bio = read_jsonl_file("Llama-1-7B-facts.jsonl")

with open("Llama-1-7B.jsonl") as f:
    batch_data = []
    for line in tqdm(f):
        dp = json.loads(line)
        batch_data.append(dp)
        if len(batch_data) == 2:
            process_data_multi(batch_data)
            batch_data = []
    if batch_data:
        process_data_multi(batch_data)

3550it [05:34, 10.61it/s]  
